This notebook handles downloading, loading, parsing, joining, and saving the combined PLUTO and Rolling Sales Data datasets.

Sources:
* [Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)
* [PLUTO Data](http://www1.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page)

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import zipfile
import io

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)

Rolling sales data is provided as lightly formatted `xls` files (Excel):

![alt text](./rolling-sales-data-excel-screencap.png "Logo Title Text 1")

Luckily the extraneous details are easily patched up post-import.

In [3]:
rolling_sales_data = dict()
rolling_sales_data_key_pairs = {'Manhattan': 'manhattan',
                                'Brooklyn': 'brooklyn',
                                'Queens': 'queens',
                                'Bronx': 'bronx',
                                'Staten Island': 'statenisland'}
for b_k, b_xls in tqdm(list(rolling_sales_data_key_pairs.items())):
    borough_rsd = pd.read_excel('https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/rollingsales_{0}.xls'.format(b_xls))
    borough_rsd.columns = borough_rsd.iloc[3].values
    borough_rsd = borough_rsd[4:]
    rolling_sales_data[b_k] = borough_rsd

100%|██████████| 5/5 [00:22<00:00,  4.44s/it]


PLUTO data is provided as borough-denominated `csv` files packaged into a `zip`. The following code bit unpacks the data and rekeys the file (`QN.csv`, `BK.csv`, `BX.csv`, `SI.csv`, `Mn.csv`) to match the lexicon used for the rolling sales data, above.

In [4]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_15v1.zip')
pluto_key_pairs = {'Manhattan': 'Mn.csv',
                   'Brooklyn': 'BK.csv',
                   'Bronx': 'BX.csv',
                   'Staten Island': 'SI.csv',
                   'Queens': 'QN.csv'}
pluto_data = dict()
for b_k, b_csv in tqdm(list(pluto_key_pairs.items())):
    with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
        borough_pluto = pd.read_csv(ar.open(b_csv))
        pluto_data[b_k] = borough_pluto

  0%|          | 0/5 [00:00<?, ?it/s]/Users/Honors/anaconda/envs/residential-housing/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4,6,7,8,10,11,50,52,53,77,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 20%|██        | 1/5 [00:03<00:12,  3.05s/it]/Users/Honors/anaconda/envs/residential-housing/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6,7,8,10,27,50,52,53,77,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 60%|██████    | 3/5 [00:12<00:07,  3.68s/it]/Users/Honors/anaconda/envs/residential-housing/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6,7,10,11,50,52,53,77,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interac

The data is provided on a per-borough basis, and since we would like to study the entire city, we must now flatten each set of tables into two big tables. Along the way we encode an additional `Borough` column, to preserve information.

**Note**: the `Borough` mapping is a new column in the rolling sales dataset; a prexisting (numerically encoded) `BOROUGH` column is removed. The mapping overlays and replaces an older (acronym-encoded) column in the `PLUTO` one.

In [5]:
rolling_sales_agglom = pd.DataFrame(columns=rolling_sales_data['Manhattan'].columns)
pluto_data_agglom = pd.DataFrame(columns=pluto_data['Manhattan'].columns)
for b_k in tqdm(pluto_key_pairs.keys()):
    pluto_data[b_k]['Borough'] = rolling_sales_data[b_k]['Borough'] = b_k
    rolling_sales_agglom = pd.concat([rolling_sales_agglom, rolling_sales_data[b_k]], ignore_index=True)
    pluto_data_agglom = pd.concat([pluto_data_agglom, pluto_data[b_k]], ignore_index=True)
del rolling_sales_agglom['BOROUGH']

100%|██████████| 5/5 [00:11<00:00,  2.49s/it]


Now we flatten these two files into one by performing an outer join on the `(Borough, Block, Lot)` unique key (this is [standard](http://www1.nyc.gov/nyc-resources/service/1232/borough-block-lot-bbl-lookup)).

In order to do this we must first remap the column names in the rolling sales dataset from `SPACED ALL CAPS` to `NoSpaceCamelCase` (as ued by `PLUTO`).

In [6]:
rolling_sales_agglom.columns = [c.title().replace(' ', '') for c in list(rolling_sales_agglom.columns)]

At this point we hit a roadblock.

Our assumption going into this project was that the `Borough`-`Block`-`Lot` columns present in both PLUTO and the Rolling Sales dataset (these are an identifier used for taxation purposes) are equivalent, allowing us to use this combo as a unique key for a join. However, this assumption turns out to be incorrect.

The Rolling Sales dataset contains information on the sale of both individual apartments and of whole buildings, but does not contain the square footage of the apartment sales in the data.

The PLUTO dataset contains information on the square footage of entire buildings, but not on that of individual apartments. This is because instead of using the usual Tax-Block-Lot system PLUTO defines and uses its own Lot configuration, painting over condominiums with multiple lots in a single building by merging them into one Lot. This is useful for geospatial visualization but voids the use of the dataset for ordinary apartment-based residential lookup: the Rolling Sales dataset contains sales information on individual apartments which, as a consequence of this tweak, cannot be mapped to any of the records in PLUTO.

We'll have to remove apartments from the dataset and do without them. Weirdly the sentinel value in this case is `'            '`, as in, a long space.

In [7]:
rolling_sales_agglom.ix[0, :]['ApartmentNumber']

'            '

In [8]:
rs_a_f = rolling_sales_agglom[rolling_sales_agglom['ApartmentNumber'] == '            ']

Not every record in the Rolling Sales dataset represents an actual sale. A large number of records are of what are effectively deed transfers: sales of a building for either `$0` or occassionally some paltry some, usually between family members, sometimes as a part of a contract, and so on. Since these records do not encode any actual information they should be dropped.

`$1000` is arbitrarily chosen as a cutoff value.

In [9]:
rs_a_f = rs_a_f[rs_a_f['SalePrice'] > 1000]

In addition to the primary key values there are a handful of columns which are present in both datasets. However, these are encoded somewhat differently. For example, `Address` is present in both `PLUTO` and the rolling sales data, but is not consistently formatted in the former&mdash;many entries have what appears to be a leading space that would need to be stripped first.

Considering that the alphanumerical `Borough-Block-Lot` combination is standardly encoded and fully unique (*for our chosen subcase*), we can simply not consider these additional columns, as adding them to the join won't do anything for us (and create more problems than it solves).

The rolling sales data contains a number of variables which are more or less copies of the data contained in `PLUTO`. Since we cannot construct a generalized classifier based on variables which are not everywhere present (technically you can `GLM`-encode `NaN` values, but this is a poor idea), we will extract only one column of interest from the rolling sales data, the `SalePrice`.

In [10]:
rolling_non_dups = ['Borough', 'Block', 'Lot', 'SalePrice']

Now the join.

In [11]:
rolling_pluto = pd.merge(rs_a_f[rolling_non_dups], pluto_data_agglom,
                         how='outer', on=['Borough', 'Block', 'Lot'])

It's not immediately apparent why, but this resulted in more records than expected. I chose to defer investigating this until a second stage of the process.

In [12]:
len(rolling_pluto) - len(pluto_data_agglom)

14472

However `SalePrice`-populated mergers fired correctly.

In [13]:
len(rs_a_f) - len(rolling_pluto[rolling_pluto['SalePrice'] >= 0])

0

The target variable for this analysis is `BldgArea`, so a couple more adjustments: let's drop entries missing this value or having a `BldgArea` of `0` and drop entries missing an `Address` (parks and the like).

In [14]:
rolling_pluto = rolling_pluto[rolling_pluto['BldgArea'] > 0]
rolling_pluto = rolling_pluto[rolling_pluto['Address'].notnull()]

At this point we are ready to split the dataset into two partitions. The *predictor partition* contains all of our records with associated sales data&mdash;the ground truths on which we will build our model. The *predicted partition* contains all of the fresh records (the vast majority) which do not have sales data associated with them.

Once we are satisfied with the power of our model (constructed using the predictor data) we will apply it to the rest of the city (predicted data) and visualize the results for explanatory analysis.

In [15]:
r_p_pre = rolling_pluto[rolling_pluto['SalePrice'].notnull()]
r_p_post = rolling_pluto[rolling_pluto['SalePrice'].isnull()]

Let's enrich the dataset with a calculated variable for value by square footage.

In [16]:
# Ignore the warning.
sqft_values = r_p_pre['SalePrice'] / r_p_pre['BldgArea']
r_p_pre['ValueSqFt'] = sqft_values
r_p_post['ValueSqFt'] = np.nan

/Users/Honors/anaconda/envs/residential-housing/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/Honors/anaconda/envs/residential-housing/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


The index appears to be some combination out of wack.

In [21]:
r_p_pre.reset_index(drop=True, inplace=True)
r_p_post.reset_index(drop=True, inplace=True)

That does it!

In [29]:
r_p_pre.to_csv('nyc_building_sales.csv')
r_p_post.to_csv('nyc_buildings.csv')